In [32]:
# IMPORT LIBRARY
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import roc_auc_score

# LOAD DATA & CLEAN
df = pd.read_csv('credit_scoring.csv')
df_clean = df.drop(columns=['leak_col_good', 'leak_col_subtle'])
print (df)


      application_id  age  monthly_income  loan_amount  previous_defaults  \
0             501000   41        13995609      5982664                  0   
1             501001   58        13683833      3711198                  0   
2             501002   33         9417391      7172332                  0   
3             501003   45         6861811      8661056                  0   
4             501004   22         5640742      4520669                  1   
...              ...  ...             ...          ...                ...   
5995          506995   26        18253018      3690565                  2   
5996          506996   40         9944385      4082038                  1   
5997          506997   28         9048532     14760384                  0   
5998          506998   51         6843160     11774594                  0   
5999          506999   35        12715733      8006387                  0   

      credit_score  default  leak_col_good  leak_col_subtle  
0            

In [ ]:
# SPLIT DATA
X = df_clean.drop(columns=['application_id', 'default'])
y = df_clean['default']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [31]:
# Logistic Regression
logreg = LogisticRegression(max_iter=1000)
logreg.fit(X_train, y_train)
proba_log = logreg.predict_proba(X_test)[:, 1]
auc_log = roc_auc_score(y_test, proba_log)

# Gradient Boosting
gb = GradientBoostingClassifier()
gb.fit(X_train, y_train)
proba_gb = gb.predict_proba(X_test)[:, 1]
auc_gb = roc_auc_score(y_test, proba_gb)

# Skor 300–850 dari probabilitas
def to_score(p, bawah=300, atas=850):
    return (1 - p) * (atas - bawah) + bawah

score_gb = to_score(proba_gb)

# Hasil
print(f"AUC Logistic: {auc_log:.4f}")
print(f"AUC Gradien Boosting : {auc_gb:.4f}")

pd.DataFrame({
    'application_id': df_clean.loc[y_test.index, 'application_id'],
    'default_prob': proba_gb,
    'score': score_gb.round()
})


AUC Logistic: 0.7990
AUC Gradien Boosting : 0.7038


,application_id,default_prob,score
1782,502782,0.008490,845.0
3917,504917,0.002204,849.0
221,501221,0.008265,845.0
2135,503135,0.031186,833.0
5224,506224,0.001093,849.0
...,...,...,...
2543,503543,0.062206,816.0
1773,502773,0.001924,849.0
2846,503846,0.000759,850.0
5799,506799,0.007594,846.0
